In [1]:
import os
import cv2
import shutil
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
from sklearn.model_selection import train_test_split

from src.config import *
from src.data import *
from src.models.efficientnet import EfficientNetB5Custom
from src.utils import *
from src.data import OriginalOAIDataset
from src.train import train, train_model
from src.trainers.classification import Classification


np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)




In [6]:
MENDELEY_EXPERT1_SPLIT_PATH
AUG_MENDELEY_EXPERT1_SPLIT_PATH = 'dataset/mendeley_dataset/augmented_mendeley_expert1_split'


In [3]:
img = explorar_split_data(MENDELEY_EXPERT1_SPLIT_PATH)

╒═════════╤═════╤═════╤═════╤═════╤═════╕
│ Clase   │   0 │   1 │   2 │   3 │   4 │
╞═════════╪═════╪═════╪═════╪═════╪═════╡
│ train   │ 359 │ 333 │ 162 │ 154 │ 144 │
├─────────┼─────┼─────┼─────┼─────┼─────┤
│ val     │  51 │  47 │  23 │  22 │  20 │
├─────────┼─────┼─────┼─────┼─────┼─────┤
│ test    │ 104 │  97 │  47 │  45 │  42 │
╘═════════╧═════╧═════╧═════╧═════╧═════╛


In [8]:

ORIGINAL_TRAIN_PATH = os.path.join(MENDELEY_EXPERT1_SPLIT_PATH, 'train')
classes = os.listdir(ORIGINAL_TRAIN_PATH)
print(classes)
data_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

NEW_TRAIN_PATH = os.path.join(AUG_MENDELEY_EXPERT1_SPLIT_PATH, 'train')
NEW_VAL_PATH = os.path.join(AUG_MENDELEY_EXPERT1_SPLIT_PATH, 'val')
NEW_TEST_PATH = os.path.join(AUG_MENDELEY_EXPERT1_SPLIT_PATH, 'test')

# Create new directories
if not os.path.exists(AUG_MENDELEY_EXPERT1_SPLIT_PATH):
    os.makedirs(AUG_MENDELEY_EXPERT1_SPLIT_PATH)
if not os.path.exists(NEW_TRAIN_PATH):
    os.makedirs(NEW_TRAIN_PATH)
if not os.path.exists(NEW_VAL_PATH):
    os.makedirs(NEW_VAL_PATH)
if not os.path.exists(NEW_TEST_PATH):
    os.makedirs(NEW_TEST_PATH)
for split in ['train', 'val', 'test']:
    SPLIT_PATH = os.path.join(AUG_MENDELEY_EXPERT1_SPLIT_PATH, split)
    for class_name in classes:
        CLASS_PATH = os.path.join(SPLIT_PATH, class_name)
        if not os.path.exists(CLASS_PATH):
            os.makedirs(CLASS_PATH)



['0', '1', '2', '3', '4']


In [ ]:
for class_name in classes:
    CLASS_PATH = os.path.join(NEW_TRAIN_PATH, class_name)
    class_origin = os.path.join(ORIGINAL_TRAIN_PATH, class_name)
    num_augmentations = 1000 - len(os.listdir(class_origin))
    print(f'Augmenting {num_augmentations} images for class {class_name}')

    # Copy original images
    for img_name in os.listdir(class_origin):
        img_path = os.path.join(class_origin, img_name)
        img = cv2.imread(img_path)
        
        # Verificar si la imagen fue leída correctamente
        if img is None:
            print(f"Error al leer la imagen {img_path}. Puede que no sea una imagen válida o esté dañada.")
            continue
        
        
        # Copiar la imagen
        new_img_path = os.path.join(CLASS_PATH, f"{class_name}_{img_name}")
        cv2.imwrite(new_img_path, img)

    print(f"Se han copiado {len(os.listdir(CLASS_PATH))} imágenes de la clase {class_name}")
    print(f"Generando imágenes aumentadas para la clase {class_name}...")
    
    while(len(os.listdir(CLASS_PATH)) < 1000):
        
        probabilidad = ((1000 - len(os.listdir(CLASS_PATH))) / len(os.listdir(class_origin)))
        for img_name in os.listdir(class_origin):
            if len(os.listdir(CLASS_PATH)) >= 1000:
                break
            if np.random.rand() > probabilidad:
                continue
            
            img_path = os.path.join(class_origin, img_name)
            img = cv2.imread(img_path)
            img_array = img.reshape((1, ) + img.shape)
            for batch in data_gen.flow(img_array, batch_size=1, save_to_dir=CLASS_PATH, save_prefix='aug', save_format='png'):
                break
            
            

    print(f"Se han generado {len(os.listdir(CLASS_PATH))} imágenes aumentadas para la clase {class_name}\n-----------------------------------\n")

Augmenting 641 images for class 0
Se han copiado 359 imágenes de la clase 0
Generando imágenes aumentadas para la clase 0...
Probabilidad: 1.8355153203342618
Probabilidad: 0.45252454417952315
Probabilidad: 0.20874855156431055
Probabilidad: 0.10596620908130941
Probabilidad: 0.06214574898785426
Se han generado 1000 imágenes aumentadas para la clase 0
-----------------------------------

Augmenting 667 images for class 1
Se han copiado 333 imágenes de la clase 1
Generando imágenes aumentadas para la clase 1...
Probabilidad: 2.0530030030030026
Probabilidad: 0.5605740181268882
Probabilidad: 0.23343195266272188
Probabilidad: 0.14529025191675793
Probabilidad: 0.10596620908130941
Probabilidad: 0.07880658436213991
Se han generado 1000 imágenes aumentadas para la clase 1
-----------------------------------

Augmenting 838 images for class 2
Se han copiado 162 imágenes de la clase 2
Generando imágenes aumentadas para la clase 2...
Probabilidad: 5.222839506172839
Probabilidad: 2.1459752321981425
P

In [16]:
explorar_split_data(AUG_MENDELEY_EXPERT1_SPLIT_PATH)
mostrar_imagenes(AUG_MENDELEY_EXPERT1_SPLIT_PATH)

╒═════════╤══════╤══════╤══════╤══════╤══════╕
│ Clase   │    0 │    1 │    2 │    3 │    4 │
╞═════════╪══════╪══════╪══════╪══════╪══════╡
│ train   │ 1000 │ 1000 │ 1000 │ 1000 │ 1000 │
├─────────┼──────┼──────┼──────┼──────┼──────┤
│ val     │    0 │    0 │    0 │    0 │    0 │
├─────────┼──────┼──────┼──────┼──────┼──────┤
│ test    │    0 │    0 │    0 │    0 │    0 │
╘═════════╧══════╧══════╧══════╧══════╧══════╛
No se encontraron imágenes en los subdirectorios.
